### Libraries

In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import h5py

### Helper Functions

In [2]:
def open_file(filename, filepath):
    """
    Opens the pickled dataframe stored at the specified location.
    ---
    IN: string
    OUT: pandas dataframe
    """
    with open(filepath + filename, 'rb') as picklefile: 
        df = pickle.load(picklefile)
        
    if df.empty:
        print(f"Dataframe at {filename} is empty.")
    
    return df

### Sample Stock & Wikipedia Data by Week

Need to improve this so it takes the open on Monday and close on Friday, not the mean of five opens and closes.

In [3]:
filepath_stock_wiki='/Users/Joe/Documents/Metis/Projects/metis-two-Luther/data/combined-wiki-price-data/'
stock_files = os.listdir(filepath_stock_wiki)

In [4]:
# Must write a custom function to get the opening and closing value

In [5]:
df = open_file(stock_files[10], filepath_stock_wiki)

In [6]:
df.set_index('date', drop=True, inplace=True)

In [7]:
#df['close'].resample("W").first()

In [8]:
aggregation_functions = {'size (bytes)': [sum],
                         'size_delta': [sum],
                         'minor_edit': [sum],
                         'edit_count': [sum],
                         'high': [np.max],
                         'low':  [np.min],
                         'volume': [sum],
                         'adj_high': [np.max],
                         'adj_low':  [np.min],
                         'adj_volume': [sum]}

In [9]:
def sample_by_week(filename, agg_func = aggregation_functions):
    # Open dataframe
    filepath_stock_wiki='/Users/Joe/Documents/Metis/Projects/metis-two-Luther/data/combined-wiki-price-data/'
    df = open_file(filename, filepath_stock_wiki)
    
    # Sample by Week
    df.set_index('date', drop=True, inplace=True)
    df_weekly = df.resample("W").agg(agg_func)
    
    # For opening and closing values, take the first and last of the week, respectively
    opening = df['open'].resample("W").first()
    closing = df['close'].resample("W").last()
    opening_adj = df['adj_open'].resample("W").first()
    closing_adj = df['adj_close'].resample("W").last()
    
    df_weekly.columns = df_weekly.columns.droplevel(1)
    df_weekly = df_weekly.join([opening, closing, opening_adj, closing_adj])
    return df_weekly

### Create Stock Targets

In [10]:
def create_stock_targets(df): 
    # Non-Minor Edit
    df['non-minor_edit'] = df['edit_count'] - df['minor_edit']
    
    # Create Stock Targets
    df['percent_change'] = (df['close'] - df['open'])/df['open']
    df['swing'] = (df['high'] - df['low'])/df['open']
    
    df['percent_adj_change'] = (df['adj_close'] - df['adj_open'])/df['adj_open']
    df['adj_swing'] = (df['adj_high'] - df['adj_low'])/df['adj_open']
    
    return df

### Open and Format Trend Data

#### Google quantifies search quatities as such:  

Numbers represent search interest relative to the highest point on the chart for the given region and time. A value of 100 is the peak popularity for the term. A value of 50 means that the term is half as popular. Likewise a score of 0 means the term was less than 1% as popular as the peak.

In [11]:
filepath_trends = '/Users/Joe/Documents/Metis/Projects/metis-two-Luther/data/google-trends/'
trend_files = [file for file in os.listdir(filepath_trends) if file.startswith('.') == False]

In [12]:
def trend_dataframe(filename):
    # Load Data
    filepath_trends = '/Users/Joe/Documents/Metis/Projects/metis-two-Luther/data/google-trends/'
    df_trend = pd.read_csv(filepath_trends + filename)
    
    # Format Data
    df_trend = df_trend.iloc[1:]
    df_trend.rename(columns={'Category: All categories': 'search_interest'}, inplace=True)
    df_trend.index = pd.to_datetime(df_trend.index)
    
    return df_trend

### Merging Datasets  
Conveniently, they both increment on Sunday.

In [13]:
def trend_filename(wiki_filename):
    abbrev = wiki_filename.replace('wikipedia-and-stock-history-', '').replace('.pkl', '')
    return abbrev + '.csv'

In [14]:
def abbrev_from_filename(wiki_filename):
    abbrev = wiki_filename.replace('wikipedia-and-stock-history-', '').replace('.pkl', '')
    return abbrev

In [15]:
def merge_wiki_and_trend(wiki_filename):
    # Open Wiki/Stock df and resample by week
    try:
        df_wiki = sample_by_week(wiki_filename, agg_func = aggregation_functions)
    except:
        print(f"Error on {wiki_filename}")
        raise
    
    # Create Stock Targets
    df_wiki = create_stock_targets(df_wiki)
    
    # Get the name of trends file from the abbrev in the stock/wiki filename
    trend_csv = trend_filename(wiki_filename)
    try:
        df_trend = trend_dataframe(trend_csv)
        df = pd.merge(df_wiki, df_trend, how='inner', left_index=True, right_index=True)
        return df
    except:
        # There is no trend data for this firm
        print(f'Collect trend data for {trend_csv}')
        return None

In [16]:
# Compression
def save_file(df, abbrev):
    filepath = '/Users/Joe/Documents/Metis/Projects/metis-two-Luther/data/stock_wiki_trend_dataframes/'
    compressor = 'blosc'
    
    filename = filepath + abbrev + '.h5'
    df.to_hdf(filename, 'table', mode='w', complevel=9, complib=compressor)

In [17]:
# Merge and save every company collected
for file in stock_files:
    df = open_file(file, filepath_stock_wiki)
    if df.empty:
        # There is no data here. Skip this step.
        continue
    
    abbrev = abbrev_from_filename(file)
    df = merge_wiki_and_trend(file)
    try:
        save_file(df, abbrev)
    except:
        print(f"Not saving a merged dataframe for {abbrev}")
        pass

Collect trend data for BBBY.csv
Not saving a merged dataframe for BBBY
Dataframe at wikipedia-and-stock-history-CTRP.pkl is empty.
Dataframe at wikipedia-and-stock-history-FLEX.pkl is empty.
Dataframe at wikipedia-and-stock-history-JD.pkl is empty.
Dataframe at wikipedia-and-stock-history-LBTYK.pkl is empty.
Dataframe at wikipedia-and-stock-history-NTES.pkl is empty.
Dataframe at wikipedia-and-stock-history-NXPI.pkl is empty.
Dataframe at wikipedia-and-stock-history-QVCA.pkl is empty.
Dataframe at wikipedia-and-stock-history-SHPG.pkl is empty.
Collect trend data for URBN.csv
Not saving a merged dataframe for URBN


#### This gives us combined Wikipedia, Google Trend, and Stock data for 112 companies